In [1]:
import parso

In [2]:
module_content = """
import foo
import bar.baz
from blub import bla

class Reverse:
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]
"""

def get_module_tree(module_content):
    return parso.parse(module_content)

tree = get_module_tree(module_content)

In [3]:
def get_class_definitions(module_tree):
    """Yields meta data about function definitions in a Python modules parse tree
    as tuples (name, start line, end line)."""
    line_idx = 0
    for c in module_tree.iter_classdefs():
        yield (c.name.value, c.start_pos[line_idx], c.end_pos[line_idx])
    
class_definitions = [c for c in get_class_definitions(tree)]
class_definitions

[('Reverse', 6, 19)]

In [4]:
def get_function_definitions(module_tree):
    """Yields meta data about function definitions in a Python modules parse tree
    as tuples (name, start line, end line)."""
    line_idx = 0
    for f in module_tree.iter_funcdefs():
        yield (f.name.value, f.start_pos[line_idx], f.end_pos[line_idx])

function_definitions = [c for c in get_function_definitions(tree)]
function_definitions

[('reverse', 20, 23)]

In [5]:
imports = [i for i in tree.iter_imports()]
imports

[<ImportName: import foo@2,0>,
 <ImportName: import bar.baz@3,0>,
 <ImportFrom: from blub import bla@4,0>]

In [6]:
def get_imports(module_tree):
    for i in module_tree.iter_imports():
        if isinstance(i, parso.python.tree.ImportFrom):
            for n in i.get_from_names():
                yield n.value
        else:
            for n in i.get_defined_names():
                yield n.value

In [7]:
imports = [i for i in get_imports(tree)]
imports

['foo', 'bar', 'blub']